* **Student**: Minhchau Dang
* **Email Address**: minhchau.dang@berkeley.edu
* **Course**: 2016-0111 DATASCI W261: Machine Learning at Scale
* **Section**: Spring 2016, Section 2
* **Assignment**: Homework 13, Week 13
* **Submission Date**: April 21, 2016

# Helper utilities

## Set Cluster Size

Using r3.xlarge nodes, there is 23424 MB of available memory for Yarn scheduling and there are 4 cores per server.

* http://www.ec2instances.info
* http://docs.aws.amazon.com/ElasticMapReduce/latest/ReleaseGuide/emr-hadoop-task-config.html

In [ ]:
node_count = 9
server_cores = 4
yarn_memory = 23424
executors_per_node = 1

## Set Cluster Name

In [ ]:
cluster_name = 'Week13_GraphX'

# HW 13.3: Spark GraphX versus your implementation of PageRank

First, we'll want to test it against our test data. However, the data format for GraphX requires integer values and our test data consists of letters. So, we'll need to transform it.

In [ ]:
!cat PageRank-test.txt | sed 's/A/1/g' | sed 's/B/2/g' | sed 's/C/3/g' | \
    sed 's/D/4/g' | sed 's/E/5/g' | sed 's/F/6/g' | sed 's/G/7/g' | sed 's/H/8/g' | \
    sed 's/I/9/g' | sed 's/J/10/g' | sed 's/K/11/g' > PageRank-testint.txt

!hdfs dfs -copyFromLocal PageRank-testint.txt

Now we run page rank on the test data set.

## Setup Scala Project

In [ ]:
%%writefile PageRank.scala
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.graphx.lib._
import org.apache.spark.rdd.RDD

/**
 * Retrieve edges from a single line in the input file
 */
def getEdges(line:String): List[Edge[Int]] = {
  val splitLine = line.split("\t")

  val label:String = splitLine(0)
  val re = "'([^']*)'".r

  val matchList = (re findAllIn splitLine(1)).matchData.toList
  matchList.map { matchItem => Edge(label.toLong, matchItem.group(1).toLong, 1) }
}

val edgesRDD = sc.textFile("PageRank-testint.txt").flatMap(getEdges)
val dataRDD:Graph[Any,Int] = Graph.fromEdges(edgesRDD, 1)
val pageRankRDD = PageRank.run(dataRDD, numIter=40, resetProb=0.15).vertices

println pageRankRDD.collect()

Writing PageRank.scala


## Submit Scala job

Before we run the shell script, we'll make sure that our log4j.properties suppresses INFO level logging so that we only print out WARN or above.

In [ ]:
!sudo sed -i 's/INFO/WARN/g' /usr/lib/spark/conf/log4j.properties

Now we submit our script to the Spark shell. Given that we are just using the test set, we don't need to provide the extra configuration options for the shell. We'll use the options mentioned here.

http://spark.apache.org/docs/latest/quick-start.html

In [ ]:
!spark-shell < PageRank.scala

/bin/sh: scala: command not found
